In [ ]:
# key words:
# savings = constant
# none = reserved for total parent

In [ ]:
!pip install plotly

In [ ]:
400 // 365

1

In [ ]:
hell = "hell"
hello = hell
hello += "v"
print(hell, hello)

hell hellv


In [ ]:
A = 2
B =A
B += 1
print(A,B)

2 3


In [ ]:
hello = "hellolo"
hello[1:]

'ellolo'

In [3]:
((5+6)+3)%3

2

In [150]:
import pandas as pd
import math
import plotly.express as px
import matplotlib.pyplot as plt
 
class CashFlow:
    def __init__(self, gc, gsheet:str): 
        self.gsheet = gsheet
        self.raw_inputnodes, self.raw_indextree = self.import_sheet(gc, gsheet)
        self.indexdict = self.build_index_tree(self.raw_indextree, self.raw_inputnodes)
        self.valuedict = self.build_input_nodes(self.raw_inputnodes)
        self.epoch = self.valuedict['epoch']['startdate']
        self.daymonth = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        self.actual = self.import_actual() 
    
### Indexing and Import Methods
    def import_actual(self) -> pd.DataFrame():
        actual = gc.open_by_url(gsheet)
        aread = actual.worksheet("Actual Transactions").get_all_values()
        df = pd.DataFrame(aread[1:], columns=aread[0])
        df['epoch time'] = 0
        # df['epoch time'] = df['Date'].apply(lambda x: self.date_to_epoch(self.format_date(x)))
        df.sort_values(by="epoch time", inplace=True)
        df['$CAD'] = pd.to_numeric(df['$CAD'])
        return df

    def format_date(self, rd:str) -> str:
        date = rd.split("/")
        ret = ""
        for c in date:
            if len(c) < 2: c = "0"+c
            ret += c
        return ret 

    def import_sheet(self, gc, gsheet:str) -> tuple:
        wb = gc.open_by_url(gsheet)
        read_inputnodes = wb.worksheet('InputNodes').get_all_values()
        read_indextree = wb.worksheet('IndexTree').get_all_values()
 
        raw_inputnodes = pd.DataFrame(read_inputnodes[1:], columns=read_inputnodes[0])
        raw_inputnodes['value'] = raw_inputnodes['value'].apply(lambda x: x.replace(',',''))
        # gspread imports everything as string, converting value and timeframe columns to numeric
        raw_inputnodes = raw_inputnodes.drop(columns=['value']).join(raw_inputnodes[['value']].apply(pd.to_numeric))
 
        raw_indextree = pd.DataFrame(read_indextree[1:], columns=read_indextree[0])
        return raw_inputnodes, raw_indextree
 
    def build_input_nodes(self, raw_inputnodes:pd.DataFrame()) -> dict:
        retdict = {}
        for i in range(raw_inputnodes.shape[0]):
            current = raw_inputnodes.iloc[i]
            retdict[current['name'].replace(' ', '')] = {col:current[col] for col in current.index if col != 'name'}
        return retdict
 
    def build_index_tree(self, raw_indextree:pd.DataFrame(), raw_inputnodes:pd.DataFrame()) -> dict:
        retdict = {}
        for i in range(raw_indextree.shape[0]):
            current = raw_indextree.iloc[i]
            children = raw_indextree[raw_indextree['parent'] == current['name']]
            node_children = raw_inputnodes[raw_inputnodes['parent'] == current['name']]
            
            if children.shape[0] == 0 and node_children.shape[0] == 0:
                retdict[current['name']] = 0
                continue
 
            children = children['name'].tolist()+node_children['name'].tolist()
            retdict[current['name']] = list(map(lambda x: x.strip(), children)) #cleans up any mistaken spaces        
        return retdict  

### Calculation Engine  
    def sum_all(self, name:str, inc_dates, sd:int, ed:int) -> None:
        # checking if the name/section is a leaf
        if name not in self.indexdict.keys():
            if name == 'savings': # alredy calculated savings
                return 
            else: 
                # calculating payment
                self.calculate_transactions(name, inc_dates, sd, ed)
            return 
        
        # iterating to next step down
        for next in self.indexdict[name]:
            self.sum_all(next, inc_dates, sd, ed)

    def calculate_transactions(self, name:str, inc_dates:dict, sd:int, ed:int) -> None:
        # calling the payment object
        payment_obj = self.valuedict[name]

        # initalizing start and end ranges
        range_start, range_end = self.initialize_date_range(payment_obj, sd, ed)
        
        # return the function if the payment is out of range
        if range_start == -1: return
        # convert to hashmap of functions?

        # calculating total payments in timeframe, based on occurence
        if payment_obj['occurrence'].strip().lower() == 'once':
            self.pay(self.date_to_epoch(payment_obj['startdate']), float(payment_obj['value']), inc_dates)

        elif payment_obj['occurrence'].strip().lower() == 'daily':
            # incrementing each day in the range
            for date in range(range_start, range_end, 1):
                # incrementing the payment on a certain date
                self.pay(date, float(payment_obj['value']), inc_dates)
        
        elif payment_obj['occurrence'].strip().lower() == 'weekly':
            # incrementing every 7 days (weekly)
            for date in range(range_start, range_end, 7):
                # incrementing the payment on a certain date
                self.pay(date, float(payment_obj['value']), inc_dates)
        
        elif payment_obj['occurrence'].strip().lower() == 'bi-weekly':
            # incrementing every 7 days (weekly)
            for date in range(range_start, range_end, 14):
                # incrementing the payment on a certain date
                self.pay(date, float(payment_obj['value']), inc_dates)
        
        elif payment_obj['occurrence'].strip().lower() == 'monthly':
            # generating dates
            for date in self.generate_monthly(payment_obj, sd, ed):
                # incrementing the payment on a certain date
                self.pay(date, float(payment_obj['value']), inc_dates)
        else: 
            raise Exception("Incorrect occurence type encountered")

    def initialize_date_range(self, payment_obj:dict, sd:int, ed:int) -> tuple:
        # converting the payment start and end dates to epoch time
        paystart = self.date_to_epoch(payment_obj['startdate'])
        payend = self.date_to_epoch(payment_obj['enddate'])
        
        # initializing export variables
        sout = eout = 0

        # checking if the payment is relevant (within the epoch, and has payments in the desired timeframe)
        if 0 < paystart <= ed: sout = paystart
        elif paystart < 0: raise Exception('EPOCH ERROR: Payment evaluated before epoch') 
        elif paystart > ed: sout = -1

        # checking if the payment ends before the end of the epoch
        if 0 < payend < ed: eout = payend
        elif payend < 0: raise Exception('EPOCH ERROR: Payment evaluated before epoch') 
        else: eout = ed
        
        # returning export variables
        return sout, eout

    def pay(self, edate:int, amount:float, inc_dates:dict) -> None:
        # checking if a day increment already exists for this date, otherwise create new entry
        if edate in inc_dates:
            inc_dates[edate] += amount
        else:
            inc_dates[edate] = amount
        inc_dates[edate] = round(inc_dates[edate], 2)

    def generate_monthly(self, paymentobj, sd:int, ed:int) -> float:
        # initalizing iterable
        # converting back to string for current month
        currentdate = self.epoch_to_date(sd)
        
        # keeping track of the exact payment date
        startday = int(self.epoch_to_date(sd)[2:4])

        # while the current date is <= than the enddate
        while self.date_to_epoch(currentdate) <= ed:
            # yielding the current day for payment
            yield(self.date_to_epoch(currentdate))
            
            # if the payday is <= 28
            if int(currentdate[2:4]) <= 28:            
                # calculating the difference
                delta = self.daymonth[(int(currentdate[:2])-1)%12]  

                # incrementing the current month
                currentdate = self.date_increment(currentdate, delta)
            else:
                delta = 0
                # checking if the date occurs in the next month 
                next = self.daymonth[(int(currentdate[:2]))%12]
                if startday > next:
                    delta = next
                else: 
                    delta = self.daymonth[(int(currentdate[:2])-1)%12]
                
                # incrementing the current month
                currentdate = self.date_increment(currentdate, delta)


                
### Simulation / Visualizer Method
    def visualize_range(self, sections:list, startdate:str, enddate:str, timeskip:int):
        # creating date ranges for visualizer
        dates = [t for t in self.generate_dates(startdate, enddate, timeskip)]
        
        # converting dates to epoch time
        sd = self.date_to_epoch(startdate)
        ed = self.date_to_epoch(enddate)

        # initializing list of dataframes for concatenation
        data_frames = []

        # creating simulational data
        for section in sections:
            # initializing schema, with savings being the first addition of money
            increment_dates = {0:float(self.valuedict['savings']['value'])}

            # populating the increment dates
            self.sum_all(section, increment_dates, sd, ed)
            # integrating the dates
            integral = self.integrate(increment_dates, sd, ed)
            # display(integral)

            # creating the dataframe for visualization
            temp = pd.DataFrame()
            temp['date'] = dates
            temp['$CAD'] = integral
            temp['section'] = section 
            data_frames.append(temp)

        # concatentation of all the dataframes
        df = pd.concat(data_frames)
        pd.set_option("display.max_rows", None, "display.max_columns", None)

        # creating the titles
        title = sections[0]
        if len(sections) > 1: title = ' vs '.join(sections)

        # generating the plot
        fig = px.line(df, x='date', y='$CAD', title=title, color='section')
        fig.update_xaxes(type='category')
        fig.show()

    def integrate(self, increment_dates:dict, sd:int, ed:int) -> list:
        # initializing the iterable
        currentsum = increment_dates[0]

        # the result of the integral 
        integral = [currentsum]        

        # summing all the increment dates at each point
        for date in range(1, ed+1):
            # if the date is an increment date, then add to the sum
            if date in increment_dates.keys():
                # incrementing the iterable
                currentsum += increment_dates[date]
            
            # appending to the integral, regardless if there's a change
            integral.append(currentsum)
        return integral
### Datetime Library
    # converting dates to epochs
    def date_to_epoch(self, date:str) -> int:
        # days from new years
        epoch_from_ny = sum(self.daymonth[:int(self.epoch[:2])-1]) + int(self.epoch[2:4])
        date_from_ny = sum(self.daymonth[:int(date[:2])-1]) + int(date[2:4])

        # calculating number of years between year of epoch and current year
        years = (365*(int(date[4:])-int(self.epoch[4:])))

        # final calculation
        converted = (date_from_ny - epoch_from_ny) + years
        if converted < 0: 
            print(date)
            raise Exception("BEFORE EPOCH ERROR: Cannot parse date before epoch")
        return converted

    # incrementing dates
    def date_increment(self, date:str, increment:int) -> str:
        # convert whole thing into days since start, then convert back
        # day 0 is new year's day of current year
        currentday = sum(self.daymonth[:(int(date[:2])-1)]) + int(date[2:4]) -1 
        currentday += increment
 
        yeardisplacement = math.floor(currentday/365)
        year = int(date[4:])+yeardisplacement
 
        currentday %= 365   
 
        monthsums = [currentday-sum(self.daymonth[:x]) for x in range(12) if currentday-sum(self.daymonth[:x]) >= 0]
        day = min(monthsums)+1
        month = monthsums.index(min(monthsums))+1
 
        if month < 10: month = '0'+str(month)
        if day < 10: day = '0'+str(day)
        return str(month)+str(day)+str(year)

    # converting from epoch to date
    def epoch_to_date(self, ep:int) -> str:
        # updating epoch time
        epoch_time = ep

        # days from new year's to epoch
        epoch_from_ny = sum(self.daymonth[:int(self.epoch[:2])-1]) + int(self.epoch[2:4])
        
        # subtracting difference between new years and epoch date
        epoch_time += epoch_from_ny

        # calculating the days since new years
        days_from_ny = epoch_time % 365

        # calculating the number of years
        years = epoch_time // 365

        # converting the days from NY to months
        monthsum = 0
        month = 0
        while monthsum < days_from_ny:
            monthsum += self.daymonth[month]
            month += 1
        
        # december 31st workaround
        if month < 1: monthsum = 0
        else: monthsum = sum(self.daymonth[:month-1])

        # finding the total number of days
        days = epoch_time - (365*years + monthsum)
        
        # december 31st edge-case
        if days == 0: 
            days = 31
            month = 12 

        # formatting and converting to strings
        fday = str(days)
        fmonth = str(month)
        fyear = str(2021+years)
        if days < 10:
            fday = "0"+str(days)
        if month < 10: 
            fmonth = "0"+str(month)

        return fmonth+fday+fyear

    def generate_dates(self, startdate:str, enddate:str, skip:int) -> list:
        # initializing return variable
        dates = []

        # converting start/end dates to epoch
        epocurrent = self.date_to_epoch(startdate)
        epoend = self.date_to_epoch(enddate)

        # checking if dates are valid
        if epocurrent > epoend: raise Exception("Startdate cannot be BEFORE Enddate")

        # looping while the current epoday is <= to epoend
        while epocurrent <= epoend:
            dates.append(self.epoch_to_date(epocurrent))
            epocurrent += skip

        return dates

# MISC. Display Functions
    def pretty_worker(self, name:str, recurdepth:int):
        if name not in self.indexdict.keys(): 
            print("   "*recurdepth, name, '|',self.valuedict[name]['paymenttype'])
            return 
        print("   "*recurdepth, name)
        for next in self.indexdict[name]:
            try:
                self.pretty_worker(next, recurdepth+1)
            except TypeError:
                pass
    
    def displaycosts(self):
        print('Nodes:')
        self.pretty_worker('total', 0) 

In [147]:
# authenticator
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [151]:
gsheet = 'https://docs.google.com/spreadsheets/d/1X4ahQA0pLWN3dd4X64pKShYLzDhvdM3frwBqAY7FfXY/edit?usp=sharing'
CF = CashFlow(gc, gsheet)

In [154]:
CF.visualize_range(['total','Tuition','OSAP', 'Awards&Grants'], '09012021', '01012022', 1)

In [107]:
CF.epoch_to_date(1729)

'05282026'

In [ ]:
CF.displaycosts()

Nodes:
 total
    Income
       OSAP
          OSAPGrant-Year1Term1 | single
          OSAPLoan-Year1Term1 | single
          OSAPGrant-Year1Term2 | single
          OSAPLoan-Year1Term2 | single
       Awards&Grants
          UWEntranceBursary2 | single
          UWEntranceBursary1 | single
          UWPresidentsScholarship | single
          Allowance | recurring
          SchoolCheque | single
    Expenditures
       Education
          Tuition
             Tuition-Term1A | single
             Tuition-Term1B | single
             Tuition-Term2A | single
             Tuition-Term2B | single
             Tuition-Term3A | single
             Tuition-Term3B | single
             Tuition-Term4A | single
             Tuition-Term4B | single
          Books | single
       Living
          Residence
             REVDorm-Year1Term1 | single
             REVDorm-Year1Term2 | single
          MealPlan | recurring
          Food | recurring
          Rent | recurring
       Entertainment
      